In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from textblob import TextBlob

from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [60]:
directory = '~/PycharmProjects/tfm_hugopobil'

btc_usd_grouped = pd.read_csv(f'{directory}/data/btc_usd_grouped.csv')
tweets_grouped = pd.read_csv(f'{directory}/data/tweets_grouped.csv')

In [61]:
df_clean = pd.read_csv(f'{directory}/data/tweets_clean.csv')
df_clean.head()

,index,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,1545110,Muhammad Sa'ad,NaN,Dota 2,2021-08-12 09:52:26,8.0,1089.0,1403.0,False,2021-10-28 12:30:44,"Airdrop is live , dont be late sir :)\n\n\n\n\...","['Airdrop', 'Airdrops', 'Airdropinspector', 'B...",Twitter for Android,False
1,1431064,Kikan4444,NaN,share the positive vibes of the universe 🧲. 💜 ...,2021-07-09 15:46:05,346.0,214.0,1534.0,False,2021-10-22 05:47:45,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,"['Klever', 'BTC', 'ETH', 'BNB', 'TRX']",Twitter for Android,False
2,1070632,Kripto Tiger 🇹🇷,NaN,#bsc #bscgem #gem #shrew #gmrfinance #nftart,2020-11-07 07:37:33,164.0,298.0,5577.0,False,2021-08-16 09:58:44,Easy money with \n\n bsc bnb BinanceSmartChain...,"['bsc', 'bnb', 'BinanceSmartChain', 'binance',...",Twitter Web App,False
3,1276832,Sheesh | @NanoDogecoin | INDC,NaN,#Staking #INDC!\n#Stake #NanoDogecoin & #earn:...,2011-02-10 04:39:28,151.0,453.0,6228.0,False,2021-10-19 02:09:26,NanoDogecoin is the King👑of the BSC Blockchain...,"['NanoDogecoin', 'King', 'BSC', 'Blockchain', ...",Twitter for Android,False
4,1980309,CoinMarketDaddy (CMD),United Kingdom,CMD stands for Coin Market Daddy & Was Built B...,2018-04-19 02:19:06,2833.0,1.0,118.0,False,2021-12-30 16:03:28,"Bitcoin News Roundup for June 17, 2020 cryptoc...","['cryptocurrencies', 'bitcoin', 'crypto', 'cry...",SocialBee.io v2,False


In [62]:
df_clean.dropna(subset=['hashtags'], inplace=True)
df = df_clean[['text']]
df.columns = ['tweets']
df.head()

,tweets
0,"Airdrop is live , dont be late sir :)\n\n\n\n\..."
1,something Big is coming \n$KLV Klever 💜🌟\n\nBT...
2,Easy money with \n\n bsc bnb BinanceSmartChain...
3,NanoDogecoin is the King👑of the BSC Blockchain...
4,"Bitcoin News Roundup for June 17, 2020 cryptoc..."


In [63]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
stop_words = nltk.corpus.stopwords.words(['english'])

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [64]:
lem = WordNetLemmatizer()

In [65]:
def cleaning(data):
    tweet_without_url = re.sub(r'http\S+',' ', data)
    tweet_without_hashtag = re.sub(r'#\w+', ' ', tweet_without_url)
    tweet_without_mentions = re.sub(r'@\w+',' ', tweet_without_hashtag)
    precleaned_tweet = re.sub('[^A-Za-z]+', ' ', tweet_without_mentions)

    # Tokenization
    tweet_tokens = TweetTokenizer().tokenize(precleaned_tweet)
    tokens_without_punc = [w for w in tweet_tokens if w.isalpha()]
    tokens_without_sw = [t for t in tokens_without_punc if t not in stop_words]

    text_cleaned = [lem.lemmatize(t) for t in tokens_without_sw]

    return " ".join(text_cleaned)

In [66]:
df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
df['date'] = df_clean['date']
df['date_clean'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df.drop(columns='date',inplace=True)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2555527066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweets'] = df['tweets'].apply(lambda x: cleaning(x))
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2555527066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df_clean['date']
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2555527066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,tweets,cleaned_tweets,date_clean
0,"Airdrop is live , dont be late sir :)\n\n\n\n\...",Airdrop live dont late sir Airdrop Airdrops Ai...,2021-10-28
1,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,something Big coming KLV Klever BTC ETH BNB TR...,2021-10-22
2,Easy money with \n\n bsc bnb BinanceSmartChain...,Easy money bsc bnb BinanceSmartChain binance g...,2021-08-16
3,NanoDogecoin is the King👑of the BSC Blockchain...,NanoDogecoin King BSC Blockchain DeFi altcoins...,2021-10-19
4,"Bitcoin News Roundup for June 17, 2020 cryptoc...",Bitcoin News Roundup June cryptocurrencies bit...,2021-12-30


In [67]:
def get_Subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def get_Polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

In [68]:
btc_usd_grouped = btc_usd_grouped.set_index('Date')

In [69]:
btc_usd_grouped

,Adj Close
Date,
2021-02-05,38144.308594
2021-02-06,39266.011719
2021-02-07,38903.441406
2021-02-08,46196.464844
2021-02-09,46481.105469
...,...
2022-01-24,36654.328125
2022-01-25,36954.003906
2022-01-26,36852.121094


In [70]:
def btc_price_cate(score):
    if score < 1:
        return 'negative'
    elif score == 1:
        return 'neutral'
    else:
        return 'positive'


def observe_period(period):
    res = btc_usd_grouped['Adj Close'].shift(period)/btc_usd_grouped['Adj Close']
    res = res.apply(btc_price_cate)
    return res

In [71]:
time_sentiment = observe_period(30)

In [72]:
# CHECK
df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2522087232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crypto_sentiment'] = df.date_clean.apply(lambda x: time_sentiment[x] if x in time_sentiment else np.nan)


In [73]:
df['subjectivity'] = df['cleaned_tweets'].apply(get_Subjectivity)
df['polarity'] = df['cleaned_tweets'].apply(get_Polarity)
print(df.shape)
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/816313075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subjectivity'] = df['cleaned_tweets'].apply(get_Subjectivity)


(23317, 6)


/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/816313075.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = df['cleaned_tweets'].apply(get_Polarity)


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity
0,"Airdrop is live , dont be late sir :)\n\n\n\n\...",Airdrop live dont late sir Airdrop Airdrops Ai...,2021-10-28,negative,0.550000,-0.081818
1,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,something Big coming KLV Klever BTC ETH BNB TR...,2021-10-22,negative,0.100000,0.000000
2,Easy money with \n\n bsc bnb BinanceSmartChain...,Easy money bsc bnb BinanceSmartChain binance g...,2021-08-16,negative,0.833333,0.433333
3,NanoDogecoin is the King👑of the BSC Blockchain...,NanoDogecoin King BSC Blockchain DeFi altcoins...,2021-10-19,negative,0.733333,0.433333
4,"Bitcoin News Roundup for June 17, 2020 cryptoc...",Bitcoin News Roundup June cryptocurrencies bit...,2021-12-30,positive,0.000000,0.000000


In [74]:
def getSentiment(score, threshold=0.5):
    if score < 0:
        return 'negative'
    elif 0.001 < score < threshold:
        return 'neutral'
    else:
        return 'positive'

df['sentiment'] = df['polarity'].apply(getSentiment)
df['target'] = df['sentiment'] == df['crypto_sentiment']
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2582751764.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['polarity'].apply(getSentiment)
/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_21043/2582751764.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['sentiment'] == df['crypto_sentiment']


,tweets,cleaned_tweets,date_clean,crypto_sentiment,subjectivity,polarity,sentiment,target
0,"Airdrop is live , dont be late sir :)\n\n\n\n\...",Airdrop live dont late sir Airdrop Airdrops Ai...,2021-10-28,negative,0.550000,-0.081818,negative,True
1,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,something Big coming KLV Klever BTC ETH BNB TR...,2021-10-22,negative,0.100000,0.000000,positive,False
2,Easy money with \n\n bsc bnb BinanceSmartChain...,Easy money bsc bnb BinanceSmartChain binance g...,2021-08-16,negative,0.833333,0.433333,neutral,False
3,NanoDogecoin is the King👑of the BSC Blockchain...,NanoDogecoin King BSC Blockchain DeFi altcoins...,2021-10-19,negative,0.733333,0.433333,neutral,False
4,"Bitcoin News Roundup for June 17, 2020 cryptoc...",Bitcoin News Roundup June cryptocurrencies bit...,2021-12-30,positive,0.000000,0.000000,positive,True


In [75]:
df.target.value_counts(normalize=True)

False    0.702192
True     0.297808
Name: target, dtype: float64

In [76]:
df.crypto_sentiment.value_counts()

negative    11980
positive    10487
Name: crypto_sentiment, dtype: int64

In [77]:
df.sentiment.value_counts(normalize=True)

positive    0.543295
neutral     0.349616
negative    0.107089
Name: sentiment, dtype: float64

In [78]:
df = df.set_index('date_clean')

In [79]:
df

,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target
date_clean,,,,,,,
2021-10-28,"Airdrop is live , dont be late sir :)\n\n\n\n\...",Airdrop live dont late sir Airdrop Airdrops Ai...,negative,0.550000,-0.081818,negative,True
2021-10-22,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,something Big coming KLV Klever BTC ETH BNB TR...,negative,0.100000,0.000000,positive,False
2021-08-16,Easy money with \n\n bsc bnb BinanceSmartChain...,Easy money bsc bnb BinanceSmartChain binance g...,negative,0.833333,0.433333,neutral,False
2021-10-19,NanoDogecoin is the King👑of the BSC Blockchain...,NanoDogecoin King BSC Blockchain DeFi altcoins...,negative,0.733333,0.433333,neutral,False
2021-12-30,"Bitcoin News Roundup for June 17, 2020 cryptoc...",Bitcoin News Roundup June cryptocurrencies bit...,positive,0.000000,0.000000,positive,True
...,...,...,...,...,...,...,...
2022-01-14,@ShibaTronToken This #ShibaTron Coin I like I'...,This Coin I like I waiting launch,positive,0.000000,0.000000,positive,True
2021-11-18,#linkedin #twitter #facebook #instagram #bitco...,Become member,positive,0.000000,0.000000,positive,True
2021-12-30,I'm new to Bitcoin donate if you can this is m...,I new Bitcoin donate address thank much b Liap...,positive,0.327273,0.168182,neutral,False


In [81]:
df.to_csv(f'{directory}/data/tweets_nlp_modelling.csv')